In [0]:
EVENT_HUB_NAMESPACE = "kafeelkafka.servicebus.windows.net"
TOPIC = "txn"
CONNECTION_STRING = "Endpoint=sb://kafeelkafka.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=/4sdIFVP6Iw0qGhRCgBpiBcm1Pa0I6QKM+AEhDydN3Q="

In [0]:
import socket
 
try:    socket.create_connection(("130.131.59.101", 9092), timeout=5)
except ConnectionRefusedError:
    print("Connection refused, server may not be running on localhost:9092")

In [0]:
from pyspark.sql.functions import col, expr, from_json
from pyspark.sql.types import *
from pyspark.sql.functions import current_timestamp
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger(__name__)

In [0]:
# from pyspark.sql.functions import from_json, col
# from pyspark.sql.types import * 

# schema = StructType([ 
#     StructField("Index", StringType()), 
#     StructField("Height", StringType()), 
#     StructField("Weight", DoubleType()) 
# ])

# kafka_server = "130.131.59.101:9092"
# topic_name = "txn"

# df = (
#     spark.readStream
#     .format("kafka")
#     .option("kafka.bootstrap.servers", kafka_server)
#     .option("subscribe", topic_name)
#     .option("startingOffsets", "latest")
#     .load()
# ) 

# df_json = df.selectExpr("CAST(value AS STRING)") \
#             .withColumn("data", from_json(col("value"), schema)) \
#             .select("data.*") \
 
# display(df_json)


# storage_account = "datastoragekk1"
# container = "casestudy"
# sas_token = "sp=racwdl&st=2025-08-03T14:28:30Z&se=2025-08-03T22:43:30Z&spr=https&sv=2024-11-04&sr=c&sig=ip%2BBqv5rjWH9JTFuaA5qM0A7hEGJT1prAI6Nd8%2B9oBU%3D"
# spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net",sas_token)

# delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/delta_output/"
# print("Creating delta tables....")

# # wasbs_url = f"wasbs://{container}@{storage_account}.blob.core.windows.net/{delta_path}"
# print("Delta tables created successfully")


# df_json.writeStream \
#     .format("delta") \
#     .outputMode("append") \
#     .option("checkpointLocation", f"wasbs://{container}@{storage_account}.blob.core.windows.net/checkpoints/").start(delta_path)

# parsed_df = df_json.select(
#     "Index",
#     "Height",
#     "Weight"
# )
# storage_account = "datastoragekk1"
# container = "casestudy"
# # sas_token = "sp=racwdl&st=2025-08-02T19:44:01Z&se=2025-08-03T03:59:01Z&spr=https&sv=2024-11-04&sr=c&sig=K2Fc8pRH2dSC6o4IH%2F5FbUf0ruq06wEp6d%2BoHqmQLx4%3D"
# sas_token = "sp=racwdl&st=2025-08-03T14:28:30Z&se=2025-08-03T22:43:30Z&spr=https&sv=2024-11-04&sr=c&sig=ip%2BBqv5rjWH9JTFuaA5qM0A7hEGJT1prAI6Nd8%2B9oBU%3D"
# spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net", sas_token)
# delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/bronze_layer/"
# checkpoint_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/checkpoints/"
# try:
#     logger.info(f"Starting streaming write to Delta at {delta_path}...")
#     query = parsed_df.writeStream \
#         .format("delta") \
#         .outputMode("append") \
#         .option("checkpointLocation", checkpoint_path) \
#         .start(delta_path)
#     logger.info("Streaming query started successfully. Awaiting termination...")
#     # query.awaitTermination()  # Uncomment to block execution
# except Exception as e:
#     logger.error(f"Error during streaming job: {e}", exc_info=True)

kafka_server = "130.131.59.101:9092"
topic_name = "txn"
schema = StructType() \
    .add("CustomerID", StringType()) \
    .add("Name", StringType()) \
    .add("Surname", StringType()) \
    .add("Gender", StringType()) \
    .add("Birthdate", DateType()) \
    .add("TransactionAmount", DoubleType()) \
    .add("Date", TimestampType()) \
    .add("MerchantName", StringType()) \
    .add("Category", StringType()) \
    .add("load_time", TimestampType())
logger.info("Starting Kafka readStream...")
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", kafka_server)
    .option("subscribe", topic_name)
    .option("startingOffsets", "latest")
    .load()
)
logger.info("Casting Kafka value to string and parsing JSON...")
df_json = df.selectExpr("CAST(value AS STRING)") \
            .withColumn("data", from_json(col("value"), schema)) \
            .select("data.*") \
            .withWatermark("load_time", "10 minutes")

display(df_json)

parsed_df = df_json.select(
    "CustomerID",
    "Name",
    "Surname",
    "Gender",
    "Birthdate",
    "TransactionAmount",
    "Date",
    "MerchantName",
    "Category",
    "load_time"
)
storage_account = "datastoragekk1"
container = "casestudy"
# sas_token = "sp=racwdl&st=2025-08-01T18:46:36Z&se=2025-08-07T03:01:36Z&sv=2024-11-04&sr=csig=H%2BCxWTspFO6ABF2tTH%2Bc61zSDIAU8HrNpw%2BoIuHoEkw%3D"
sas_token = "sp=racwdl&st=2025-08-03T14:28:30Z&se=2025-08-03T22:43:30Z&spr=https&sv=2024-11-04&sr=c&sig=ip%2BBqv5rjWH9JTFuaA5qM0A7hEGJT1prAI6Nd8%2B9oBU%3D"
spark.conf.set(f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net", sas_token)
delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/bronze_layer/"
checkpoint_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/checkpoints/"
try:
    logger.info(f"Starting streaming write to Delta at {delta_path}...")
    query = parsed_df.writeStream \
        .format("delta") \
        .outputMode("append") \
        .option("checkpointLocation", checkpoint_path) \
        .start(delta_path)
    logger.info("Streaming query started successfully. Awaiting termination...")
    # query.awaitTermination()  # Uncomment to block execution
except Exception as e:
    logger.error(f"Error during streaming job: {e}", exc_info=True)

CustomerID,Name,Surname,Gender,Birthdate,TransactionAmount,Date,MerchantName,Category,load_time
752859,Sean,Rodriguez,F,2002-10-20,35.47,2023-04-03T00:00:00Z,Smith-Russell,Cosmetic,null
26381,Michelle,Phelps,null,1985-10-24,2552.72,2023-07-17T00:00:00Z,"Peck, Spence and Young",Travel,null
305449,Jacob,Williams,M,1981-10-25,115.97,2023-09-20T00:00:00Z,Steele Inc,Clothing,null
988259,Nathan,Snyder,M,1977-10-26,11.31,2023-01-11T00:00:00Z,"Wilson, Wilson and Russell",Cosmetic,null
764762,Crystal,Knapp,F,1951-11-02,62.21,2023-06-13T00:00:00Z,Palmer-Hinton,Electronics,null
576539,Monica,Bartlett,F,2001-10-20,99.14,2023-08-24T00:00:00Z,"Tran, Torres and Joyce",Cosmetic,null
124681,Thomas,Shaw,null,1976-10-26,145.94,2023-08-30T00:00:00Z,"Evans, Griffin and Torres",Cosmetic,null
521807,Kelsey,Pena,F,1968-10-28,161.39,2023-02-08T00:00:00Z,Miller PLC,Cosmetic,null
504238,Denise,Porter,M,1957-10-31,77.73,2023-01-12T00:00:00Z,Jackson-Morgan,Restaurant,null
583532,Alexander,Richardson,F,1974-10-27,142.85,2023-06-07T00:00:00Z,"Blake, Mays and Anderson",Clothing,null


In [0]:
%pip install great-expectations

2025-08-03 15:15:26,378 INFO Received command c on object id p0


2025-08-03 15:15:27,378 INFO Received command c on object id p0


2025-08-03 15:15:28,378 INFO Received command c on object id p0


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


2025-08-03 15:15:29,378 INFO Received command c on object id p0


In [0]:
storage_account = "datastoragekk1"
container = "casestudy"

storage_sas = "sp=racwdl&st=2025-08-03T14:28:30Z&se=2025-08-03T22:43:30Z&spr=https&sv=2024-11-04&sr=c&sig=ip%2BBqv5rjWH9JTFuaA5qM0A7hEGJT1prAI6Nd8%2B9oBU%3D"

spark.conf.set(
    f"fs.azure.sas.{container}.{storage_account}.blob.core.windows.net",
    storage_sas
)

delta_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/bronze_layer/"

In [0]:
%pip install great_expectations==0.15.39

2025-08-03 15:15:40,378 INFO Received command c on object id p0


2025-08-03 15:15:41,378 INFO Received command c on object id p0


2025-08-03 15:15:42,378 INFO Received command c on object id p0


Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


2025-08-03 15:15:43,378 INFO Received command c on object id p0


In [0]:

%restart_python

2025-08-03 15:15:44,301 INFO Received command c on object id p1
2025-08-03 15:15:44,380 INFO Received command c on object id p0


In [0]:
%fs ls dbfs:/

path,name,size,modificationTime
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/geasd/,geasd/,0,1754221045000
dbfs:/local_disk0/,local_disk0/,0,1753860123000
dbfs:/mnt/,mnt/,0,1753872675000
dbfs:/volume/,volume/,0,0
dbfs:/volumes/,volumes/,0,0


In [0]:
import datetime
from pyspark.sql.functions import sha2, concat_ws, lit, DataFrame
import pandas as pd
from ruamel import yaml

from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.util import get_context
from great_expectations.data_context.types.base import (
    DataContextConfig,
    FilesystemStoreBackendDefaults,
)
from great_expectations.core.batch import RuntimeBatchRequest
import datetime
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.validation_operators.validation_operators import ValidationOperatorResult

In [0]:
bronze_df.sample(withReplacement=False, fraction=0.1).limit(10).show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8609280291829800>, line 1
----> 1 bronze_df.sample(withReplacement=False, fraction=0.1).limit(10).show()

NameError: name 'bronze_df' is not defined

In [0]:
root_directory = "/dbfs/geasd/great_expectations/"

data_context_config = DataContextConfig(
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)
context = get_context(project_config=data_context_config)


my_spark_datasource_config = {
    "name": "my_datasource",
    "class_name": "Datasource",
    "execution_engine": {
        "class_name": "SparkDFExecutionEngine"
    },
    "data_connectors": {
        "runtime_data_connector": {  
            "class_name": "RuntimeDataConnector",
            "module_name": "great_expectations.datasource.data_connector",
            "batch_identifiers": [
                "pipeline_stage",
                "run_id"
            ],
        }
    }
}

context.test_yaml_config(yaml.dump(my_spark_datasource_config))
context.add_datasource(**my_spark_datasource_config)

bronze_df = spark.read.format("delta").load(delta_path)

batch_request = RuntimeBatchRequest(
    datasource_name="my_datasource",
    data_connector_name="runtime_data_connector",
    data_asset_name="my_delta_table_asset",
    batch_identifiers={
        "pipeline_stage": "dev",
        "run_id": f"my_run_name_{datetime.date.today().strftime('%Y%m%d')}",
    },
    runtime_parameters={"batch_data": bronze_df},
)
suite_name = "brandnewsuite"

context.create_expectation_suite(suite_name, overwrite_existing=True)

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=suite_name
)

def add_expectations(validator, expectations):
    for method_name, kwargs in expectations:
        getattr(validator, method_name)(**kwargs)

expectations = [
    ("expect_column_values_to_be_between", {"column": "TransactionAmount", "min_value": 0.0}),
    ("expect_column_values_to_not_be_null", {"column": "TransactionAmount"}),
    ("expect_column_values_to_be_between", {"column": "Birthdate", "min_value": datetime.date(1900, 1, 1), "max_value": datetime.date(2012, 1, 1)}),
    ("expect_column_values_to_not_be_null", {"column": "CustomerID"}),
    ("expect_column_values_to_be_in_set", {"column": "Gender", "value_set": ["M", "F"], "mostly":1.0}),
    ("expect_column_values_to_be_of_type", {"column": "Date", "type_": "TimestampType"}),
    ("expect_column_values_to_not_be_null", {"column": "Date"}),
]

add_expectations(validator, expectations)

validator.save_expectation_suite()


suite_after_save = context.get_expectation_suite(suite_name)
print(f"Expectations after save: {len(suite_after_save.expectations)}")
for e in suite_after_save.expectations:
    print(f"- {e.expectation_type} on column '{e.kwargs.get('column', 'N/A')}'")

validation_result = validator.validate()

print(validation_result)
print(context.list_expectation_suites())
print(validation_result.success)
print(validation_result.statistics)
     

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SparkDFExecutionEngine
Data Connectors:
	runtime_data_connector:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-8609280291829804>, line 32
     29 context.test_yaml_config(yaml.dump(my_spark_datasource_config))
     30 context.add_datasource(**my_spark_datasource_config)
---> 32 bronze_df = spark.read.format("delta").load(delta_path)
     34 batch_request = RuntimeBatchRequest(
     35     datasource_name="my_datasource",
     36     data_connector_name="runtime_data_connector",
   (...)
     42     runtime_parameters={"batch_data": bronze_df},
     43 )
     44 suite_name = "brandnewsuite"

NameError: name 'delta_path' is not defined

In [0]:
# checkpoint = SimpleCheckpoint(
#     name="hhh_cp",
#     data_context=context,
# )

# checkpoint_result = checkpoint.run(
#     validations=[
#         {
#             "batch_request": batch_request,
#             "expectation_suite_name": suite_name,
#         }
#     ]
# )

# print(batch_request)


# validation_results = checkpoint_result.list_validation_results()

# for i, result in enumerate(validation_results):
#     print(f"\nValidation Result {i + 1}")
#     print("-" * 50)
#     print(f"Success: {result.success}")
#     print(f"Batch ID: {result.meta.get('batch_id')}")
#     print(f"Expectation Suite Name: {result.meta.get('expectation_suite_name')}")
#     print(f"Evaluated Expectations: {result.statistics['evaluated_expectations']}")
#     print(f"Successful Expectations: {result.statistics['successful_expectations']}")
#     print(f"Unsuccessful Expectations: {result.statistics['unsuccessful_expectations']}")
#     print(f"Success %: {round(result.statistics['success_percent'], 2)}%\n")

#     for e in result.results:
#         expectation_type = e.expectation_config.expectation_type
#         column = e.expectation_config.kwargs.get("column", "N/A")
#         success = e.success
#         print(f"  - {expectation_type} on column '{column}': {' Passed' if success else ' Failed'}")
#     print("-" * 50)

#     print(context.list_expectation_suites())